In [260]:
import pandas as pd
pd.set_option('display.max_columns', 50000)
import numpy as np
import csv
import json
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn
import datetime, os

In [261]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

Num GPUs Available:  1


In [262]:
"""
A weighted version of categorical_crossentropy for keras (2.0.6). This lets you apply a weight to unbalanced classes.
@url: https://gist.github.com/wassname/ce364fddfc8a025bfab4348cf5de852d
@author: wassname
"""
from keras import backend as K
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [263]:
df = pd.read_csv("./dataset/train.csv")
df.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,17,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,2
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,1
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,5
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,5
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,5


In [264]:
df.describe()

,age,days_since_last_login,avg_time_spent,avg_transaction_value,points_in_wallet,churn_risk_score
count,36992.000000,36992.000000,36992.000000,36992.000000,33549.000000,36992.000000
mean,37.118161,-41.915576,243.472334,29271.194003,686.882199,3.463397
std,15.867412,228.819900,398.289149,19444.806226,194.063624,1.409661
min,10.000000,-999.000000,-2814.109110,800.460000,-760.661236,-1.000000
25%,23.000000,8.000000,60.102500,14177.540000,616.150000,3.000000
50%,37.000000,12.000000,161.765000,27554.485000,697.620000,4.000000
75%,51.000000,16.000000,356.515000,40855.110000,763.950000,5.000000
max,64.000000,26.000000,3235.578521,99914.050000,2069.069761,5.000000


In [265]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   customer_id                   36992 non-null  object 
 1   Name                          36992 non-null  object 
 2   age                           36992 non-null  int64  
 3   gender                        36992 non-null  object 
 4   security_no                   36992 non-null  object 
 5   region_category               31564 non-null  object 
 6   membership_category           36992 non-null  object 
 7   joining_date                  36992 non-null  object 
 8   joined_through_referral       36992 non-null  object 
 9   referral_id                   36992 non-null  object 
 10  preferred_offer_types         36704 non-null  object 
 11  medium_of_operation           36992 non-null  object 
 12  internet_option               36992 non-null  object 
 13  l

In [266]:
def valueCounts(df):
    print(df["gender"].value_counts())
    print("-"*50)
    print(df["region_category"].value_counts())
    print("-"*50)
    print(df["membership_category"].value_counts())
    print("-"*50)
    print(df["joined_through_referral"].value_counts())
    print("-"*50)
    print(df["preferred_offer_types"].value_counts())
    print("-"*50)
    print(df["medium_of_operation"].value_counts())
    print("-"*50)
    print(df["internet_option"].value_counts())
    print("-"*50)
    print(df["used_special_discount"].value_counts())
    print("-"*50)
    print(df["offer_application_preference"].value_counts())
    print("-"*50)
    print(df["past_complaint"].value_counts())
    print("-"*50)
    print(df["complaint_status"].value_counts())
    print("-"*50)
    print(df["feedback"].value_counts())
    print("-"*50)
    print(df["avg_frequency_login_days"].value_counts())
    print("-"*50)
    print(df["churn_risk_score"].value_counts())
    print("-"*50)
    
valueCounts(df)

F          18490
M          18443
Unknown       59
Name: gender, dtype: int64
--------------------------------------------------
Town       14128
City       12737
Village     4699
Name: region_category, dtype: int64
--------------------------------------------------
Basic Membership       7724
No Membership          7692
Gold Membership        6795
Silver Membership      5988
Premium Membership     4455
Platinum Membership    4338
Name: membership_category, dtype: int64
--------------------------------------------------
No     15839
Yes    15715
?       5438
Name: joined_through_referral, dtype: int64
--------------------------------------------------
Gift Vouchers/Coupons       12349
Credit/Debit Card Offers    12274
Without Offers              12081
Name: preferred_offer_types, dtype: int64
--------------------------------------------------
Desktop       13913
Smartphone    13876
?              5393
Both           3810
Name: medium_of_operation, dtype: int64
-------------------------

In [267]:
df.isnull().values.any()

True

In [268]:
df.isnull().sum()

customer_id                        0
Name                               0
age                                0
gender                             0
security_no                        0
region_category                 5428
membership_category                0
joining_date                       0
joined_through_referral            0
referral_id                        0
preferred_offer_types            288
medium_of_operation                0
internet_option                    0
last_visit_time                    0
days_since_last_login              0
avg_time_spent                     0
avg_transaction_value              0
avg_frequency_login_days           0
points_in_wallet                3443
used_special_discount              0
offer_application_preference       0
past_complaint                     0
complaint_status                   0
feedback                           0
churn_risk_score                   0
dtype: int64

In [269]:
# setting target wrong value -1 to 1 assuming sign issue, 
df['churn_risk_score'] = df['churn_risk_score'].apply(lambda x:1 if x == -1 else x)
df['churn_risk_score'] = df['churn_risk_score'].apply(lambda x:x-1)

In [270]:
def handleMissingValues(df):
    
    df['joining_date']  = pd.to_datetime(df['joining_date'],format="%Y-%m-%d")
    
    
    # replacing Unknown words with most occured in df for gender column
    df['gender'] = df['gender'].replace(['Unknown'], df["gender"].mode()[0])
    df['joined_through_referral'] = df['joined_through_referral'].replace(['?'], df["joined_through_referral"].mode()[0])
    df['avg_frequency_login_days'] = df['avg_frequency_login_days'].replace(['Error'], None)
    
    
    df["region_category"] = df["region_category"].fillna(df["region_category"].mode()[0])
    df["preferred_offer_types"] = df["preferred_offer_types"].fillna(df["preferred_offer_types"].mode()[0])
    df["points_in_wallet"] = df["points_in_wallet"].fillna(df["points_in_wallet"].median())
    df["avg_frequency_login_days"] = df["avg_frequency_login_days"].fillna(df["avg_frequency_login_days"].median())
    
    
    df["avg_frequency_login_days"] = pd.to_numeric(df["avg_frequency_login_days"])
    
    return df

df = handleMissingValues(df)
df.isnull().sum()

customer_id                     0
Name                            0
age                             0
gender                          0
security_no                     0
region_category                 0
membership_category             0
joining_date                    0
joined_through_referral         0
referral_id                     0
preferred_offer_types           0
medium_of_operation             0
internet_option                 0
last_visit_time                 0
days_since_last_login           0
avg_time_spent                  0
avg_transaction_value           0
avg_frequency_login_days        0
points_in_wallet                0
used_special_discount           0
offer_application_preference    0
past_complaint                  0
complaint_status                0
feedback                        0
churn_risk_score                0
dtype: int64

In [271]:
valueCounts(df)

F    18549
M    18443
Name: gender, dtype: int64
--------------------------------------------------
Town       19556
City       12737
Village     4699
Name: region_category, dtype: int64
--------------------------------------------------
Basic Membership       7724
No Membership          7692
Gold Membership        6795
Silver Membership      5988
Premium Membership     4455
Platinum Membership    4338
Name: membership_category, dtype: int64
--------------------------------------------------
No     21277
Yes    15715
Name: joined_through_referral, dtype: int64
--------------------------------------------------
Gift Vouchers/Coupons       12637
Credit/Debit Card Offers    12274
Without Offers              12081
Name: preferred_offer_types, dtype: int64
--------------------------------------------------
Desktop       13913
Smartphone    13876
?              5393
Both           3810
Name: medium_of_operation, dtype: int64
--------------------------------------------------
Wi-Fi          1

In [272]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   customer_id                   36992 non-null  object        
 1   Name                          36992 non-null  object        
 2   age                           36992 non-null  int64         
 3   gender                        36992 non-null  object        
 4   security_no                   36992 non-null  object        
 5   region_category               36992 non-null  object        
 6   membership_category           36992 non-null  object        
 7   joining_date                  36992 non-null  datetime64[ns]
 8   joined_through_referral       36992 non-null  object        
 9   referral_id                   36992 non-null  object        
 10  preferred_offer_types         36992 non-null  object        
 11  medium_of_operation         

In [273]:
def handleExtraFeatures(df):

    from datetime import date
    df['todays_date'] = date.today().strftime("%Y-%m-%d")
    df['todays_date']  = pd.to_datetime(df['todays_date'],format="%Y-%m-%d")
    

    df['customer_days'] = df['todays_date'] - df['joining_date'] 
    df['customer_days'] = df['customer_days'] / np.timedelta64(1,'D')

    return df
df = handleExtraFeatures(df)
df.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,todays_date,customer_days
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,17,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,1,2021-03-27,1318.0
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,2017-08-28,No,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10.0,697.62,Yes,No,Yes,Solved,Quality Customer Care,0,2021-03-27,1307.0
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,4,2021-03-27,1597.0
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,4,2021-03-27,1610.0
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,4,2021-03-27,1292.0


In [274]:
def labelEncoding(df, onlyTransform: False, label_object = {}):
    categorical_columns = [
        'gender',
        'joined_through_referral',
        'used_special_discount',
        'offer_application_preference',
        'past_complaint'
    ]
    for col in categorical_columns:
        if col in df.columns:
            labelencoder = None
            if onlyTransform is False:
                labelencoder = LabelEncoder()
                labelencoder.fit(df[col])
                label_object[col] = labelencoder
            else:
                labelencoder = label_object[col]

            df[col] = labelencoder.transform(df[col])
    return df, label_object

df, label_object = labelEncoding(df, onlyTransform= False, label_object= {})
df.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,todays_date,customer_days
0,fffe4300490044003600300030003800,Pattie Morrisey,18,0,XW0DQ7H,Village,Platinum Membership,2017-08-17,0,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,17,300.63,53005.25,17.0,781.75,1,1,0,Not Applicable,Products always in Stock,1,2021-03-27,1318.0
1,fffe43004900440032003100300035003700,Traci Peery,32,0,5K0N3X1,City,Premium Membership,2017-08-28,0,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10.0,697.62,1,0,1,Solved,Quality Customer Care,0,2021-03-27,1307.0
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,0,1F2TCL3,Town,No Membership,2016-11-11,1,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22.0,500.69,0,1,1,Solved in Follow-up,Poor Website,4,2021-03-27,1597.0
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,1,VJGJ33N,City,No Membership,2016-10-29,1,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6.0,567.66,0,1,1,Unsolved,Poor Website,4,2021-03-27,1610.0
4,fffe43004900440031003900350030003600,Meghan Kosak,31,0,SVZXCWB,City,No Membership,2017-09-12,0,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16.0,663.06,0,1,1,Solved,Poor Website,4,2021-03-27,1292.0


In [275]:
def encodeMembership(df):
    
    cleanup_nums = {
                    "membership_category":{
                        "No Membership":0,
                        "Basic Membership":1,
                        "Gold Membership":2,
                        "Silver Membership":3,
                        "Premium Membership":4,
                        "Platinum Membership":5
                    }
                   }
    
    df = df.replace(cleanup_nums)
    
    return df

df = encodeMembership(df)
df.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,todays_date,customer_days
0,fffe4300490044003600300030003800,Pattie Morrisey,18,0,XW0DQ7H,Village,5,2017-08-17,0,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,17,300.63,53005.25,17.0,781.75,1,1,0,Not Applicable,Products always in Stock,1,2021-03-27,1318.0
1,fffe43004900440032003100300035003700,Traci Peery,32,0,5K0N3X1,City,4,2017-08-28,0,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10.0,697.62,1,0,1,Solved,Quality Customer Care,0,2021-03-27,1307.0
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,0,1F2TCL3,Town,0,2016-11-11,1,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22.0,500.69,0,1,1,Solved in Follow-up,Poor Website,4,2021-03-27,1597.0
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,1,VJGJ33N,City,0,2016-10-29,1,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6.0,567.66,0,1,1,Unsolved,Poor Website,4,2021-03-27,1610.0
4,fffe43004900440031003900350030003600,Meghan Kosak,31,0,SVZXCWB,City,0,2017-09-12,0,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16.0,663.06,0,1,1,Solved,Poor Website,4,2021-03-27,1292.0


In [276]:
def oneHotEncoding(df, onlyTransform=False, one_hot_object = {}):
    categorical_columns = [
        'region_category',
        'preferred_offer_types',
        'medium_of_operation',
        'internet_option',
        'complaint_status',
        'feedback'
    ]
    for col in categorical_columns:
        
        col_data  = df[col].values.reshape(-1,1)
        
        if onlyTransform is False:
            onehot_encoder = OneHotEncoder()
            onehot_encoder = onehot_encoder.fit(col_data)
            one_hot_object[col] = onehot_encoder
        
        onehot_encoded = one_hot_object[col].transform(col_data).toarray()
        columns_list = []
        for i in range(onehot_encoded.shape[1]):
            columns_list.append(col+"_"+str(int(i)))
        dfOneHot = pd.DataFrame(onehot_encoded, columns = columns_list) 
        df = pd.concat([df, dfOneHot], axis=1)
        

    df = df.drop(categorical_columns,axis=1)
    return df, one_hot_object


df, one_hot_object = oneHotEncoding(df, onlyTransform=False, one_hot_object = {})
df.head()

,customer_id,Name,age,gender,security_no,membership_category,joining_date,joined_through_referral,referral_id,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,churn_risk_score,todays_date,customer_days,region_category_0,region_category_1,region_category_2,preferred_offer_types_0,preferred_offer_types_1,preferred_offer_types_2,medium_of_operation_0,medium_of_operation_1,medium_of_operation_2,medium_of_operation_3,internet_option_0,internet_option_1,internet_option_2,complaint_status_0,complaint_status_1,complaint_status_2,complaint_status_3,complaint_status_4,feedback_0,feedback_1,feedback_2,feedback_3,feedback_4,feedback_5,feedback_6,feedback_7,feedback_8
0,fffe4300490044003600300030003800,Pattie Morrisey,18,0,XW0DQ7H,5,2017-08-17,0,xxxxxxxx,16:08:02,17,300.63,53005.25,17.0,781.75,1,1,0,1,2021-03-27,1318.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,fffe43004900440032003100300035003700,Traci Peery,32,0,5K0N3X1,4,2017-08-28,0,CID21329,12:38:13,16,306.34,12838.38,10.0,697.62,1,0,1,0,2021-03-27,1307.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,0,1F2TCL3,0,2016-11-11,1,CID12313,22:53:21,14,516.16,21027.00,22.0,500.69,0,1,1,4,2021-03-27,1597.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,1,VJGJ33N,0,2016-10-29,1,CID3793,15:57:50,11,53.27,25239.56,6.0,567.66,0,1,1,4,2021-03-27,1610.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,fffe43004900440031003900350030003600,Meghan Kosak,31,0,SVZXCWB,0,2017-09-12,0,xxxxxxxx,15:46:44,20,113.13,24483.66,16.0,663.06,0,1,1,4,2021-03-27,1292.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [277]:
def removeRedundantColumns(df):
    
    redundant_columns = ['customer_id','Name','security_no','joining_date','referral_id','last_visit_time','todays_date']

    df = df.drop(redundant_columns,axis=1)
    
    return df

df = removeRedundantColumns(df)
df.head()

,age,gender,membership_category,joined_through_referral,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,churn_risk_score,customer_days,region_category_0,region_category_1,region_category_2,preferred_offer_types_0,preferred_offer_types_1,preferred_offer_types_2,medium_of_operation_0,medium_of_operation_1,medium_of_operation_2,medium_of_operation_3,internet_option_0,internet_option_1,internet_option_2,complaint_status_0,complaint_status_1,complaint_status_2,complaint_status_3,complaint_status_4,feedback_0,feedback_1,feedback_2,feedback_3,feedback_4,feedback_5,feedback_6,feedback_7,feedback_8
0,18,0,5,0,17,300.63,53005.25,17.0,781.75,1,1,0,1,1318.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,32,0,4,0,16,306.34,12838.38,10.0,697.62,1,0,1,0,1307.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,44,0,0,1,14,516.16,21027.00,22.0,500.69,0,1,1,4,1597.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,37,1,0,1,11,53.27,25239.56,6.0,567.66,0,1,1,4,1610.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,31,0,0,0,20,113.13,24483.66,16.0,663.06,0,1,1,4,1292.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [278]:
df = df.sample(frac=1).reset_index(drop=True)

In [279]:
from keras.utils import np_utils
y = pd.DataFrame(df["churn_risk_score"])
weights = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(df["churn_risk_score"].values), df["churn_risk_score"].values)
class_weights = dict(enumerate(weights))
y = np_utils.to_categorical(df["churn_risk_score"])
df = df.drop("churn_risk_score", axis=1)

X = pd.DataFrame(df)
print(X.shape)
print(y.shape)

(36992, 40)
(36992, 5)


c:\users\pitbox\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2 3 4], y=[3 4 2 ... 4 2 3] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [280]:
print(weights)

[1.93929227 2.69916089 0.70974674 0.72640157 0.75286456]


In [281]:
from imblearn.over_sampling import SMOTE
smote = SMOTE('minority')
X_sm,y_sm = smote.fit_resample(X=X,y=y)
print(X_sm.shape,y_sm.shape)

X = X_sm
y = y_sm

c:\users\pitbox\appdata\local\programs\python\python38\lib\site-packages\imblearn\utils\_validation.py:587: FutureWarning: Pass sampling_strategy=minority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


(44675, 40) (44675, 5)


c:\users\pitbox\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2 3 4] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [304]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y,shuffle=True)

X_train = X
y_train = y
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(44675, 40)
(44675, 5)
(8935, 40)
(8935, 5)


In [305]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# scalerX = StandardScaler()
scalerX = RobustScaler()
# scalerX = MinMaxScaler()
scalerX = scalerX.fit(X_train)
X_train = scalerX.transform(X_train)
X_train = pd.DataFrame(X_train)

X_test = scalerX.transform(X_test)
X_test = pd.DataFrame(X_test)

In [284]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout 
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(64,input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu',kernel_regularizer=L1L2()))
model.add(Dense(32, activation='relu',kernel_regularizer=L1L2()))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy',get_f1])

print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 64)                2624      
_________________________________________________________________
dense_22 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_23 (Dense)             (None, 5)                 165       
Total params: 4,869
Trainable params: 4,869
Non-trainable params: 0
_________________________________________________________________
None


In [285]:
class_weights

{0: 1.9392922673656618,
 1: 2.6991608901860635,
 2: 0.7097467382962395,
 3: 0.7264015709376535,
 4: 0.7528645568332146}

In [306]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=5000,epochs=300,verbose=1,shuffle=False)

Epoch 1/300
9/9 [==============================] - 0s 30ms/step - loss: 0.4445 - accuracy: 0.7952 - get_f1: 0.7926 - val_loss: 0.4899 - val_accuracy: 0.7783 - val_get_f1: 0.7751
Epoch 2/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4968 - accuracy: 0.7724 - get_f1: 0.7682 - val_loss: 0.5084 - val_accuracy: 0.7783 - val_get_f1: 0.7770
Epoch 3/300
9/9 [==============================] - 0s 18ms/step - loss: 0.5633 - accuracy: 0.7602 - get_f1: 0.7580 - val_loss: 0.5382 - val_accuracy: 0.7656 - val_get_f1: 0.7603
Epoch 4/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4903 - accuracy: 0.7800 - get_f1: 0.7778 - val_loss: 0.4914 - val_accuracy: 0.7827 - val_get_f1: 0.7803
Epoch 5/300
9/9 [==============================] - 0s 19ms/step - loss: 0.4979 - accuracy: 0.7789 - get_f1: 0.7770 - val_loss: 0.4825 - val_accuracy: 0.7850 - val_get_f1: 0.7780
Epoch 6/300
9/9 [==============================] - 0s 19ms/step - loss: 0.4921 - accuracy: 0.7755 - get_f1: 0.

Epoch 47/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4389 - accuracy: 0.7967 - get_f1: 0.7939 - val_loss: 0.4386 - val_accuracy: 0.7924 - val_get_f1: 0.7883
Epoch 48/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4392 - accuracy: 0.7969 - get_f1: 0.7940 - val_loss: 0.4384 - val_accuracy: 0.7924 - val_get_f1: 0.7888
Epoch 49/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4383 - accuracy: 0.7974 - get_f1: 0.7943 - val_loss: 0.4379 - val_accuracy: 0.7931 - val_get_f1: 0.7897
Epoch 50/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4381 - accuracy: 0.7976 - get_f1: 0.7944 - val_loss: 0.4377 - val_accuracy: 0.7933 - val_get_f1: 0.7897
Epoch 51/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4382 - accuracy: 0.7974 - get_f1: 0.7946 - val_loss: 0.4375 - val_accuracy: 0.7931 - val_get_f1: 0.7898
Epoch 52/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4377 - accuracy: 0.7977 - get_

Epoch 93/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4324 - accuracy: 0.7997 - get_f1: 0.7967 - val_loss: 0.4325 - val_accuracy: 0.7944 - val_get_f1: 0.7895
Epoch 94/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4318 - accuracy: 0.7996 - get_f1: 0.7966 - val_loss: 0.4325 - val_accuracy: 0.7936 - val_get_f1: 0.7901
Epoch 95/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4318 - accuracy: 0.7997 - get_f1: 0.7965 - val_loss: 0.4322 - val_accuracy: 0.7945 - val_get_f1: 0.7895
Epoch 96/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4317 - accuracy: 0.7996 - get_f1: 0.7967 - val_loss: 0.4324 - val_accuracy: 0.7931 - val_get_f1: 0.7899
Epoch 97/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4318 - accuracy: 0.7996 - get_f1: 0.7967 - val_loss: 0.4321 - val_accuracy: 0.7950 - val_get_f1: 0.7892
Epoch 98/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4313 - accuracy: 0.7997 - get_

9/9 [==============================] - 0s 18ms/step - loss: 0.4269 - accuracy: 0.8005 - get_f1: 0.7979 - val_loss: 0.4269 - val_accuracy: 0.7978 - val_get_f1: 0.7944
Epoch 139/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4269 - accuracy: 0.8007 - get_f1: 0.7982 - val_loss: 0.4275 - val_accuracy: 0.7978 - val_get_f1: 0.7922
Epoch 140/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4265 - accuracy: 0.8008 - get_f1: 0.7980 - val_loss: 0.4267 - val_accuracy: 0.7981 - val_get_f1: 0.7944
Epoch 141/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4266 - accuracy: 0.8007 - get_f1: 0.7982 - val_loss: 0.4273 - val_accuracy: 0.7983 - val_get_f1: 0.7919
Epoch 142/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4264 - accuracy: 0.8010 - get_f1: 0.7979 - val_loss: 0.4265 - val_accuracy: 0.7987 - val_get_f1: 0.7938
Epoch 143/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4264 - accuracy: 0.8006 - get_f1: 0.79

9/9 [==============================] - 0s 17ms/step - loss: 0.4226 - accuracy: 0.8022 - get_f1: 0.7997 - val_loss: 0.4234 - val_accuracy: 0.7987 - val_get_f1: 0.7943
Epoch 184/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4224 - accuracy: 0.8024 - get_f1: 0.7993 - val_loss: 0.4241 - val_accuracy: 0.7988 - val_get_f1: 0.7938
Epoch 185/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4226 - accuracy: 0.8021 - get_f1: 0.7998 - val_loss: 0.4233 - val_accuracy: 0.7985 - val_get_f1: 0.7950
Epoch 186/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4222 - accuracy: 0.8026 - get_f1: 0.7995 - val_loss: 0.4240 - val_accuracy: 0.7988 - val_get_f1: 0.7938
Epoch 187/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4225 - accuracy: 0.8021 - get_f1: 0.7996 - val_loss: 0.4231 - val_accuracy: 0.7990 - val_get_f1: 0.7950
Epoch 188/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4220 - accuracy: 0.8028 - get_f1: 0.79

9/9 [==============================] - 0s 17ms/step - loss: 0.4182 - accuracy: 0.8040 - get_f1: 0.8014 - val_loss: 0.4196 - val_accuracy: 0.7998 - val_get_f1: 0.7975
Epoch 229/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4188 - accuracy: 0.8040 - get_f1: 0.8016 - val_loss: 0.4194 - val_accuracy: 0.8004 - val_get_f1: 0.7980
Epoch 230/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4180 - accuracy: 0.8040 - get_f1: 0.8015 - val_loss: 0.4196 - val_accuracy: 0.8000 - val_get_f1: 0.7971
Epoch 231/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4185 - accuracy: 0.8040 - get_f1: 0.8016 - val_loss: 0.4193 - val_accuracy: 0.8003 - val_get_f1: 0.7978
Epoch 232/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4179 - accuracy: 0.8040 - get_f1: 0.8015 - val_loss: 0.4196 - val_accuracy: 0.8001 - val_get_f1: 0.7975
Epoch 233/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4186 - accuracy: 0.8042 - get_f1: 0.80

9/9 [==============================] - 0s 18ms/step - loss: 0.4149 - accuracy: 0.8062 - get_f1: 0.8040 - val_loss: 0.4163 - val_accuracy: 0.8013 - val_get_f1: 0.7981
Epoch 274/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4153 - accuracy: 0.8065 - get_f1: 0.8046 - val_loss: 0.4159 - val_accuracy: 0.8024 - val_get_f1: 0.7985
Epoch 275/300
9/9 [==============================] - 0s 17ms/step - loss: 0.4153 - accuracy: 0.8060 - get_f1: 0.8037 - val_loss: 0.4160 - val_accuracy: 0.8019 - val_get_f1: 0.7981
Epoch 276/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4155 - accuracy: 0.8065 - get_f1: 0.8044 - val_loss: 0.4155 - val_accuracy: 0.8034 - val_get_f1: 0.7982
Epoch 277/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4142 - accuracy: 0.8062 - get_f1: 0.8040 - val_loss: 0.4156 - val_accuracy: 0.8021 - val_get_f1: 0.7994
Epoch 278/300
9/9 [==============================] - 0s 18ms/step - loss: 0.4149 - accuracy: 0.8063 - get_f1: 0.80

In [307]:
prediction = model.predict(X_test)

In [308]:
y_label = tf.math.argmax(y_test, axis=1)
predict_label = tf.math.argmax(prediction,axis=1)

print(y_label)
print(predict_label)
# print(label_object['churn_risk_score'].inverse_transform(predict_label))

tf.Tensor([4 2 4 ... 1 3 3], shape=(8935,), dtype=int64)
tf.Tensor([4 2 4 ... 1 4 3], shape=(8935,), dtype=int64)


In [309]:
from sklearn.metrics import accuracy_score as sk_accuracy_score
from sklearn.metrics import f1_score as sk_f1_score
from sklearn.metrics import precision_score as sk_precision_score
from sklearn.metrics import recall_score as sk_recall_score
from sklearn.metrics import confusion_matrix as sk_confusion_matrix

print("sk_accuracy_score :",100*sk_accuracy_score(y_label, predict_label))
print("sk_f1_score :",100*sk_f1_score(y_label, predict_label,average="macro"))
print("sk_precision_score :", 100*sk_precision_score(y_label, predict_label,average="macro"))
print("sk_recall_score :" ,100*sk_recall_score(y_label, predict_label,average="macro"))

sk_accuracy_score : 80.4252937884723
sk_f1_score : 77.32594348907867
sk_precision_score : 80.07137826020558
sk_recall_score : 76.76558635281764


In [310]:
df_test = pd.read_csv("./dataset/test.csv")
df_test.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
0,fffe43004900440031003700300030003400,Alethia Meints,50,F,OQJ1XAY,Village,Premium Membership,2015-11-02,No,xxxxxxxx,Without Offers,Smartphone,Wi-Fi,07:19:30,12,386.26,40721.44,7.0,733.830000,Yes,No,No,Not Applicable,Poor Product Quality
1,fffe43004900440031003900370037003300,Ming Lopez,41,M,OUQRPKO,Village,Gold Membership,2016-03-01,No,xxxxxxxx,Without Offers,Desktop,Fiber_Optic,22:21:16,11,37.80,9644.40,9.0,726.000000,Yes,No,No,Not Applicable,Poor Website
2,fffe43004900440034003800360037003000,Carina Flannigan,31,F,02J2RE7,Town,Silver Membership,2017-03-03,No,xxxxxxxx,Gift Vouchers/Coupons,Both,Mobile_Data,16:40:39,18,215.36,3693.25,21.0,713.780000,Yes,No,Yes,Solved in Follow-up,No reason specified
3,fffe43004900440036003200370033003400,Kyung Wanner,64,M,5YEQIF1,Town,Silver Membership,2017-08-18,Yes,CID8941,Credit/Debit Card Offers,?,Fiber_Optic,14:56:17,-999,44.57,36809.56,11.0,744.970000,Yes,No,Yes,No Information Available,Too many ads
4,fffe43004900440035003000370031003900,Enola Gatto,16,F,100RYB5,Town,No Membership,2015-05-05,Yes,CID5690,Without Offers,Smartphone,Mobile_Data,02:57:53,6,349.88,40675.86,8.0,299.048351,No,Yes,Yes,Solved in Follow-up,Poor Website


In [311]:
df_test =  handleMissingValues(df_test)

In [312]:
df_test = handleExtraFeatures(df_test)

In [313]:
df_test, label_object = labelEncoding(df_test, onlyTransform= True, label_object= label_object)

In [314]:
df_test = encodeMembership(df_test)

In [315]:
df_test, one_hot_object = oneHotEncoding(df_test, onlyTransform=True, one_hot_object = one_hot_object)

In [316]:
df_test.head()

,customer_id,Name,age,gender,security_no,membership_category,joining_date,joined_through_referral,referral_id,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,todays_date,customer_days,region_category_0,region_category_1,region_category_2,preferred_offer_types_0,preferred_offer_types_1,preferred_offer_types_2,medium_of_operation_0,medium_of_operation_1,medium_of_operation_2,medium_of_operation_3,internet_option_0,internet_option_1,internet_option_2,complaint_status_0,complaint_status_1,complaint_status_2,complaint_status_3,complaint_status_4,feedback_0,feedback_1,feedback_2,feedback_3,feedback_4,feedback_5,feedback_6,feedback_7,feedback_8
0,fffe43004900440031003700300030003400,Alethia Meints,50,0,OQJ1XAY,4,2015-11-02,0,xxxxxxxx,07:19:30,12,386.26,40721.44,7.0,733.830000,1,0,0,2021-03-27,1972.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,fffe43004900440031003900370037003300,Ming Lopez,41,1,OUQRPKO,2,2016-03-01,0,xxxxxxxx,22:21:16,11,37.80,9644.40,9.0,726.000000,1,0,0,2021-03-27,1852.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,fffe43004900440034003800360037003000,Carina Flannigan,31,0,02J2RE7,3,2017-03-03,0,xxxxxxxx,16:40:39,18,215.36,3693.25,21.0,713.780000,1,0,1,2021-03-27,1485.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,fffe43004900440036003200370033003400,Kyung Wanner,64,1,5YEQIF1,3,2017-08-18,1,CID8941,14:56:17,-999,44.57,36809.56,11.0,744.970000,1,0,1,2021-03-27,1317.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,fffe43004900440035003000370031003900,Enola Gatto,16,0,100RYB5,0,2015-05-05,1,CID5690,02:57:53,6,349.88,40675.86,8.0,299.048351,0,1,1,2021-03-27,2153.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [317]:
customer_ids = df_test["customer_id"]
customer_ids_df = pd.DataFrame(data = customer_ids,columns=["customer_id"])
df_test = removeRedundantColumns(df_test)
df_test.head()

,age,gender,membership_category,joined_through_referral,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,customer_days,region_category_0,region_category_1,region_category_2,preferred_offer_types_0,preferred_offer_types_1,preferred_offer_types_2,medium_of_operation_0,medium_of_operation_1,medium_of_operation_2,medium_of_operation_3,internet_option_0,internet_option_1,internet_option_2,complaint_status_0,complaint_status_1,complaint_status_2,complaint_status_3,complaint_status_4,feedback_0,feedback_1,feedback_2,feedback_3,feedback_4,feedback_5,feedback_6,feedback_7,feedback_8
0,50,0,4,0,12,386.26,40721.44,7.0,733.830000,1,0,0,1972.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,41,1,2,0,11,37.80,9644.40,9.0,726.000000,1,0,0,1852.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,31,0,3,0,18,215.36,3693.25,21.0,713.780000,1,0,1,1485.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,64,1,3,1,-999,44.57,36809.56,11.0,744.970000,1,0,1,1317.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,16,0,0,1,6,349.88,40675.86,8.0,299.048351,0,1,1,2153.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [318]:
x_test_submit = scalerX.transform(df_test)
x_test_submit = pd.DataFrame(x_test_submit)
x_test_submit.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,0.500000,0.0,0.666667,0.0,0.000,0.741234,0.375608,-0.597044,0.233114,0.0,-1.0,0.0,0.448905,0.0,-0.787985,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.419728,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.153846,1.0,0.000000,0.0,-0.125,-0.444179,-0.700523,-0.430377,0.170151,0.0,-1.0,0.0,0.229927,0.0,-0.787985,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.419728,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-0.230769,0.0,0.333333,0.0,0.750,0.159855,-0.906598,0.569623,0.071888,0.0,-1.0,1.0,-0.439781,0.0,0.212015,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.580272,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.038462,1.0,0.333333,1.0,-126.375,-0.421149,0.240148,-0.263711,0.322692,0.0,-1.0,1.0,-0.746350,0.0,0.212015,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.580272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.807692,0.0,-0.666667,1.0,-0.750,0.617474,0.374029,-0.513711,-3.263040,-1.0,0.0,1.0,0.779197,0.0,0.212015,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-0.580272,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [319]:
prediction_submit = model.predict(x_test_submit)
predict_label = tf.math.argmax(prediction_submit,axis=1)

# predict_inversed = label_object['churn_risk_score'].inverse_transform(predict_label)
print(predict_label)
# print(predict_inversed)

tf.Tensor([2 2 2 ... 4 3 2], shape=(19919,), dtype=int64)


In [320]:
df_to_submit = pd.concat([customer_ids_df,pd.DataFrame(data = predict_label,columns=["churn_risk_score"])], axis=1)
df_to_submit['churn_risk_score'] = df_to_submit['churn_risk_score'].apply(lambda x:x+1)

In [321]:
df_to_submit.head()

,customer_id,churn_risk_score
0,fffe43004900440031003700300030003400,3
1,fffe43004900440031003900370037003300,3
2,fffe43004900440034003800360037003000,3
3,fffe43004900440036003200370033003400,3
4,fffe43004900440035003000370031003900,5


In [322]:
print(df_to_submit["churn_risk_score"].value_counts())
print("-"*50)

5    6907
3    5919
4    4080
2    1604
1    1409
Name: churn_risk_score, dtype: int64
--------------------------------------------------


In [323]:
df_to_submit.to_csv(path_or_buf="./df_to_submit"+".csv",float_format='%.0f',index=False)